In [8]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [9]:
df=pd.read_csv("../data/test_data.csv")

/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
df=df[df["label_name"]!="label"]

In [11]:
df["features2"] = df["features"].map(lambda x: [int(v) for v in x.split(",") ])

In [12]:
df.head()

,Unnamed: 0,label_name,features,features2
0,0,854,"115349,21589,1024,141917,854","[115349, 21589, 1024, 141917, 854]"
1,1,14658,"115349,21589,1024,141917,854","[115349, 21589, 1024, 141917, 854]"
2,2,854,"21589,1024,141917,854,854","[21589, 1024, 141917, 854, 854]"
3,3,14658,"21589,1024,141917,854,854","[21589, 1024, 141917, 854, 854]"
4,4,94828,"1024,141917,854,854,854","[1024, 141917, 854, 854, 854]"


In [13]:
df = df[["label_name","features"]]

In [14]:
df.to_csv("../data/test_data2.csv",index=False)

In [15]:
# The following functions can be used to convert a value to a type compatible
# with tf.Example.

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


In [16]:
!pwd

/Users/songfeng/workspace/github/learnTensorflow/src


In [17]:
def serialize_example(label, feature):
  """
  Creates a tf.Example message ready to be written to a file.
  """

  # Create a dictionary mapping the feature name to the tf.Example-compatible
  # data type.

  feature = {
      'label': _int64_feature(label),
      'feature': _int64_feature(feature)
  }

  # Create a Features message using tf.train.Example.

  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

In [20]:
serialized_example = serialize_example([94828], [1024, 141917, 854, 854, 854])

In [32]:
example_proto = tf.train.Example.FromString(serialized_example)
example_proto

features {
  feature {
    key: "feature"
    value {
      int64_list {
        value: 1024
        value: 141917
        value: 854
        value: 854
        value: 854
      }
    }
  }
  feature {
    key: "label"
    value {
      int64_list {
        value: 94828
      }
    }
  }
}

In [39]:
df["feature2"] = df["features"].map(lambda x:[ int(v) for v in x.split(",")])
df["label"] = df["label_name"].map(lambda x: int(x))

In [40]:
df.head()

,label_name,features,feature2,label
0,854,"115349,21589,1024,141917,854","[115349, 21589, 1024, 141917, 854]",854
1,14658,"115349,21589,1024,141917,854","[115349, 21589, 1024, 141917, 854]",14658
2,854,"21589,1024,141917,854,854","[21589, 1024, 141917, 854, 854]",854
3,14658,"21589,1024,141917,854,854","[21589, 1024, 141917, 854, 854]",14658
4,94828,"1024,141917,854,854,854","[1024, 141917, 854, 854, 854]",94828


In [41]:
filename= "test2.tfrecord"
with tf.python_io.TFRecordWriter(filename) as writer:
  for label,feature in zip(df["label"],df["feature2"]):
    example = serialize_example([label],feature)
    writer.write(example)